# heisernberg1D with 2 spin per site

In [5]:
from __future__ import print_function, division
import sys,os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='10' # set number of MKL threads to run in parallel
#
quspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,quspin_path)
from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator
from quspin.tools.lanczos import lanczos_full,lanczos_iter,FTLM_static_iteration,LTLM_static_iteration
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
from scipy import sparse
import scipy

In [6]:
I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

Sz1 = sparse.kron(I,Sz,format='csr')
Sz2 = sparse.kron(Sz,I,format='csr')
SzSz = sparse.kron(Sz,Sz,format='csr')

Sx1 = sparse.kron(I,Sx,format='csr')
Sx2 = sparse.kron(Sx,I,format='csr')
SxSx = sparse.kron(Sx,Sx,format='csr')

Sy1 = sparse.kron(I,Sy,format='csr')
Sy2 = sparse.kron(Sy,I,format='csr')
SySy = sparse.kron(Sy,Sy,format='csr')


h1 = -(sparse.kron(Sz1, Sz2,format='csr') 
       - sparse.kron(Sx1, Sx2,format='csr')
       - sparse.kron(Sy1, Sy2,format='csr') 
     ).real

h2 = -(sparse.kron(SzSz, np.identity(4),format='csr') 
       - sparse.kron(SxSx, np.identity(4),format='csr')
       - sparse.kron(SxSx, np.identity(4),format='csr') 
     ).real * (1/2)

h3 = -(sparse.kron(np.identity(4),SzSz, format='csr') 
       - sparse.kron(np.identity(4),SxSx, format='csr')
       - sparse.kron(np.identity(4), SySy, format='csr') 
     ).real * (1/2)

lh = h1 + h2 + h3

In [7]:
def make_local_ham(pos, hams, sps = 2, L = 10):
#     assert(len(pos) == len(hams), "size of position and hams must be the same")
#     assert(sps == hams[0].shape[0], "dimension of hamiltonian is inconsistent")
    
    cnt = 0
    h = sparse.identity(1, dtype=np.float64)
    for i in range(L):
        tmp = sparse.identity(sps)
        if i in pos:
#             print(f"insert operator at {i}th site")
            tmp = hams[cnt]
            cnt+=1
        h = sparse.kron(tmp, h, format='csr')
    return h

## construct with quspin

In [24]:
L = 8
L_ = int(L/2)
basis = spin_basis_1d(L,pauli=False)
J_zz = [[1,i,(i+1)%L] for i in range(L)] # OBC
J_xy = [[1/2.0,i,(i+1)%L] for i in range(L)] # OBC
# J_zz = [[1,2*i,(2*i+1)%L] for i in range(L_)] # OBC
# J_xy = [[1/2.0,2*i,(2*i+1)%L] for i in range(L_)] # OBC
ops_dict = dict(Jpm=[["+-",J_xy]],Jmp=[["-+",J_xy]],Jzz=[["zz",J_zz]])
H2 = quantum_operator(ops_dict,basis=basis,dtype=np.float64, check_symm=False)

Hermiticity check passed!


### check difference between energies.

In [25]:
E = H2.eigvalsh({})
# np.linalg.norm(E - E_)

In [64]:
L = 8
L_ = int(L/2)


H = sparse.csr_matrix((4**L_, 4**L_), dtype=np.float64)
O = sparse.csr_matrix((4**L_, 4**L_), dtype=np.float64)
for i in range(L_):
    tmp = make_local_ham([2*i + 1, (2*(i+1))%L], [Sz,Sz], 2, L)
    tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sx,Sx], 2, L)
    tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sy,Sy], 2, L) 
    
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sz,Sz/2], 2, L)
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sx,Sx/2], 2, L)
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sy,Sy/2], 2, L) 
    
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sz,Sz/2], 2, L)
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sx,Sx/2], 2, L)
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sy,Sy/2], 2, L)
    H += tmp
for i in range(L):
    O += make_local_ham([i], [Sz], 2, L)

In [65]:
E_exact, V_exact = scipy.linalg.eigh(H.toarray().real)
# E_.sort()
T = 1
beta = 1/T
print(f"temperature : {T}")
print(f"energy : {np.sum(E_exact * np.exp(-beta*E_exact))/np.sum(np.exp(-beta*E_exact))}")
# print(f"expectation value of Operator O : {np.trace(O.toarray()@V@np.diag(np.exp(-beta*E_))@V.T)/np.sum(np.exp(-beta*E_))}")

temperature : 1
energy : -1.6375851132825823


In [28]:
np.linalg.norm(E - E_)

2.8040763276868062e-14

### construct local hamiltonian for worm algorithm

In [29]:
L = 4
i = 0
H = make_local_ham([2*i + 1, (2*(i+1))%L], [Sz,Sz], 2, L)
H -= make_local_ham([2*i + 1, (2*(i+1))%L], [Sx,Sx], 2, L)
H -= make_local_ham([2*i + 1, (2*(i+1))%L], [Sy,Sy], 2, L) 

H += make_local_ham([2*i, (2*i+1)%L], [Sz,Sz/2], 2, L)
H -= make_local_ham([2*i, (2*i+1)%L], [Sx,Sx/2], 2, L)
H -= make_local_ham([2*i, (2*i+1)%L], [Sy,Sy/2], 2, L) 

H += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sz,Sz/2], 2, L)
H -= make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sx,Sx/2], 2, L)
H -= make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sy,Sy/2], 2, L)
np.save("array/test_model.npy", -H.real.toarray())
np.sort(scipy.linalg.eigh(H.toarray())[0])

array([-1.00000000e+00, -8.09016994e-01, -8.09016994e-01, -8.09016994e-01,
        0.00000000e+00,  0.00000000e+00,  1.11022302e-16,  1.11022302e-16,
        3.09016994e-01,  3.09016994e-01,  3.09016994e-01,  5.00000000e-01,
        5.00000000e-01,  5.00000000e-01,  5.00000000e-01,  5.00000000e-01])

In [20]:
print(H)

  (0, 0)	(0.25+0j)
  (1, 1)	(0.25+0j)
  (2, 2)	(-0.25+0j)
  (2, 4)	(-0.5+0j)
  (3, 3)	(-0.25+0j)
  (3, 5)	(-0.5+0j)
  (4, 2)	(-0.5+0j)
  (4, 4)	(-0.25+0j)
  (5, 3)	(-0.5+0j)
  (5, 5)	(-0.25+0j)
  (6, 6)	(0.25+0j)
  (7, 7)	(0.25+0j)
  (8, 8)	(0.25+0j)
  (9, 9)	(0.25+0j)
  (10, 10)	(-0.25+0j)
  (10, 12)	(-0.5+0j)
  (11, 11)	(-0.25+0j)
  (11, 13)	(-0.5+0j)
  (12, 10)	(-0.5+0j)
  (12, 12)	(-0.25+0j)
  (13, 11)	(-0.5+0j)
  (13, 13)	(-0.25+0j)
  (14, 14)	(0.25+0j)
  (15, 15)	(0.25+0j)


### check if all element is connected via worm update

In [15]:
def num2state(s, L = 4):
    state = []
    for i in range(L):
        state.append(s%4)
        s >>= 2;
    return state

def state_update(index, leg, fl):
    d = leg // 2
    l = leg % 2
    index[d] = index[d] ^ (fl << (l * 2))
    return index

num2state(53)

[1, 1, 3, 0]

In [16]:
indices = (np.array((H + 0.25*sparse.identity(H.shape[0])).nonzero()).T).tolist()
indices_list = [indices[0]]
index_rem = indices.copy()
index_bag = []
index_rem.remove(indices[0])


while indices_list:
    index = indices_list.pop()
#     print("\n\n")
#     print(index)
    index_bag.append(index)
    for i in range(4):
        for j in range(3):
            for i1 in range(4):
                for j1 in range(3):
                    tmp = state_update(index.copy(), i, j+1)
                    tmp = state_update(tmp, i1, j1+1)
                    if (tmp in indices) and (tmp not in index_bag) and (tmp not in indices_list):
                        indices_list.append(tmp)
                        if tmp in index_rem:
                            index_rem.remove(tmp)
#     print(indices_list)
print("remaining : " , index_rem)

remaining :  []


## apply unitary transformation on the same model

In [33]:
from __future__ import print_function, division
import sys,os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='10' # set number of MKL threads to run in parallel
#
quspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,quspin_path)
from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator
from quspin.tools.lanczos import lanczos_full,lanczos_iter,FTLM_static_iteration,LTLM_static_iteration
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
from scipy import sparse
import scipy

### construct unitary transformation

In [35]:
u = np.array([
    [0,1,0,0],
    [1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [-1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [0,0,0,1]
])
u = sparse.csr_matrix(u)

In [144]:
L = 6
L_ = int(L/2)


H = sparse.csr_matrix((4**L_, 4**L_), dtype=np.float64)
for i in range(L_):
    tmp = make_local_ham([2*i + 1, (2*(i+1))%L], [Sz,Sz], 2, L)
    u_tmp = make_local_ham([i, (i+1)%L_], [u, u], 4, L_)
    tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sx,Sx], 2, L)
    tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sy,Sy], 2, L) 
    
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sz,Sz/2], 2, L)
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sx,Sx/2], 2, L)
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sy,Sy/2], 2, L) 
    
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sz,Sz/2], 2, L)
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sx,Sx/2], 2, L)
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sy,Sy/2], 2, L)
    tmp = u_tmp @ tmp @ u_tmp.T

    H += tmp

In [146]:
E_st, V_st = scipy.linalg.eigh(H.toarray().real)
# E_.sort()
T = 1
beta = 1/T
print(f"temperature : {T}")
print(f"energy : {np.sum(E_st * np.exp(-beta*E_st))/np.sum(np.exp(-beta*E_st))}")
# print(f"expectation value of Operator O : {np.trace(O.toarray()@V@np.diag(np.exp(-beta*E_))@V.T)/np.sum(np.exp(-beta*E_))}")

temperature : 1
energy : -1.2322905660945869


In [66]:
np.linalg.norm(E_st - E_exact)

8.44706116973489e-15

### local hamiltonian for worm algorithm

In [149]:
L = 4
i = 0
H = make_local_ham([2*i + 1, (2*(i+1))%L], [Sz,Sz], 2, L)
H -= make_local_ham([2*i + 1, (2*(i+1))%L], [Sx,Sx], 2, L)
H -= make_local_ham([2*i + 1, (2*(i+1))%L], [Sy,Sy], 2, L) 

H += make_local_ham([2*i, (2*i+1)%L], [Sz,Sz/2], 2, L)
H -= make_local_ham([2*i, (2*i+1)%L], [Sx,Sx/2], 2, L)
H -= make_local_ham([2*i, (2*i+1)%L], [Sy,Sy/2], 2, L) 

H += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sz,Sz/2], 2, L)
H -= make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sx,Sx/2], 2, L)
H -= make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sy,Sy/2], 2, L)
u_tmp = make_local_ham([0, 1], [u, u], 4, 2)


H_transformed = H
np.save("array/test_model.npy", -H.real.toarray())
# np.sort(scipy.linalg.eigh(H.toarray())[0])

In [134]:
sign = 0
indices = sparse.find(H_transformed)
sum_ = 0
for i, j, ele in zip(indices[0], indices[1], indices[2]):
    if (i != j):
        sign += ele.real
        sum_ += np.abs(ele.real)

print(sign/sum_)

0.20049286308729863


## generate random orthogonal matrix

In [141]:
from scipy.stats import ortho_group

ave_sign_max = 0
for _ in range(int(1E5)):
    u = ortho_group.rvs(dim=4)
    u = sparse.csr_matrix(u)
    u_tmp = make_local_ham([0, 1], [u, u], 4, 2)
    
    H_unitary_tmp = u_tmp @ H @ u_tmp.T
    indices = sparse.find(-H_unitary_tmp)
    sign = 0
    sum_ = 0
    for i, j, ele in zip(indices[0], indices[1], indices[2]):
        if (i != j):
            sign += ele.real
            sum_ += np.abs(ele.real)

    if ave_sign_max < sign/sum_:
        ave_sign_max = sign/sum_
        H_unitary_cand = H_unitary_tmp
        u_cand = u
np.save("array/test_model.npy", -H_unitary_cand.real.toarray())
    

In [142]:
ave_sign_max

0.7900206072942177